# Created an interactive map for CRESST/GAIA webpage 

Goals:
- do not embed data (use APIs to load any data on-demand)
- keep it simple :) 

In [ ]:
import folium
import geopandas as gpd
import branca
from folium.elements import MacroElement
from jinja2 import Template
import xyzservices
import numpy as np
import matplotlib
import matplotlib.colors as mcolors
import branca.colormap as bcm

In [ ]:
# Restrict map to WA State
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/WA/shape.geojson"
)
minlon, minlat, maxlon, maxlat = aoi.total_bounds

## Stations

In [ ]:
m = folium.Map(
    min_lat= minlat - 1,
    max_lat= maxlat + 1,
    min_lon= minlon - 1,
    max_lon= maxlon + 1,
    max_bounds=True,
    # # NOTE: something seems to be override this...
    min_zoom=7,
    max_zoom=12,
    zoom_start=7,
    control_scale = True,
)

# Add In-Situ Stations
stations = {'Precipitation': "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/precip-stations-wa-styled.geojson",
            'Streamflow': "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/streamflow-stations-wa-styled.geojson",
            'Seismometer': "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/seismic-stations-wa-styled.geojson"}

for name, url in stations.items():

    # Get list of columns for popup
    gf = gpd.read_file(url)
    drop_cols = ['marker-size', 'marker-symbol', 'marker-color', 'geometry']
    all_fields = list(gf.drop(columns=drop_cols).columns)
    color = gf['marker-color'].iloc[0]
    hover_label = 'stid' if name != 'Seismometer' else 'station'

    mymarker = folium.Circle(fill=False, radius=200, color=color)
    gjson = folium.GeoJson(url,
                           embed=False,
                           marker=mymarker,
                           name=f"{name}").add_to(m)

    folium.features.GeoJsonTooltip(fields=[hover_label], labels=False).add_to(gjson)
    folium.features.GeoJsonPopup(fields=all_fields, labels=True).add_to(gjson);

# Basemaps

To see what is out there: https://xyzservices.readthedocs.io/en/stable/

In [ ]:
# Simplify adding tiles w/ XYZ services
xyzservices.providers.Esri#.WorldImagery

In [ ]:
# Default Basemap = CartoDB Tiles
# =============
folium.TileLayer(
    tiles=xyzservices.providers.CartoDB.Positron,
    overlay=False,
    show=True,
    control=True
).add_to(m)

# ESRI Tiles
# =============
#ESRI_API="https://server.arcgisonline.com/ArcGIS/rest/services/{variant}/MapServer/tile/{z}/{y}/{x}"
for variant in ['WorldImagery','WorldShadedRelief','WorldTopoMap','WorldGrayCanvas']:
    folium.TileLayer(
        tiles=xyzservices.providers.Esri[variant],
        name=f"ESRI {variant}",
        overlay=False,
        show=False,
        control=True
    ).add_to(m)

# NASA GIBS
# =============

# A lot of available layers, just using DEM for starters
# https://worldview.earthdata.nasa.gov
# https://www.earthdata.nasa.gov/engage/open-data-services-software/earthdata-developer-portal/gibs-api

folium.raster_layers.WmsTileLayer(
    url="https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi",
    layers="ASTER_GDEM_Greyscale_Shaded_Relief",
    name="ASTER GDEM Greyscale Shaded Relief",
    fmt="image/png",
    transparent=True,
    overlay=False,
    control=True,
    show=False,
).add_to(m)

# Example for mult-year prodcts
#years = [2023,2024,2025]
years = [2025]
for year in years:
    folium.raster_layers.WmsTileLayer(
        #url="https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi",
        url=f"https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi?TIME={year}-01-01",
        layers="OPERA_L3_DIST-ANN-HLS_Color_Index",
        name=f"OPERA Annual Disturbance {year}",
        fmt="image/png",
        #transparent=True,
        overlay=False,
        control=True,
        show=False,
        # NOT working.... need to pass in URL param directly
        #extra_wms_params={
        #    "TIME": "2023-01-01"
            #"TIME": "2024-06-01T00:00:00Z"
        #},
    ).add_to(m)

# TODO: legends for sophisticated layers
# https://gibs.earthdata.nasa.gov/wms/epsg3857/best/?request=GetMetadata&layer=OPERA_L3_DIST-ANN-HLS_Color_Index
# Apparently has a legend URL! https://gibs.earthdata.nasa.gov/legends/OPERA_Vegetation_Disturbance_Annual_H.png
# legend_url = "https://gibs.earthdata.nasa.gov/legends/OPERA_Vegetation_Disturbance_Annual_H.png"
# legend_html = f"""
# <div style="
#     position: fixed;
#     bottom: 50px; left: 50px; width: 300px; z-index:9999;
#     background-color: white; padding: 10px; border:2px solid grey;
#     box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
#     <b>OPERA Annual Disturbance 2024 Legend</b><br>
#     <img src="{legend_url}" style="width:100%;"/>
# </div>
# """
# m.get_root().html.add_child(Element(legend_html))

# WA DNR 

https://gis.dnr.wa.gov/site2/rest/services/Public_Forest_Mgmt/WADNR_PUBLIC_RS_FRIS_Rasters/MapServer/layers

https://gis.dnr.wa.gov/site2/rest/services/Public_Forest_Mgmt/WADNR_PUBLIC_RS_FRIS_Rasters/MapServer/1/tile/{z}/{y}/{x}


https://gis.dnr.wa.gov/site2/rest/services/Public_Forest_Mgmt/WADNR_PUBLIC_RS_FRIS_Rasters/MapServer/1/tile/2/0/0

In [ ]:
# TODO: revisit this... seems like tile server acts differently to others

# Map Name to layer number from here https://gis.dnr.wa.gov/site2/rest/services/Public_Forest_Mgmt/WADNR_PUBLIC_RS_FRIS_Rasters/MapServer
# BA (1) so Mapserver/1/tile ...
# folium.TileLayer(
#     tiles=(
#         "https://gis.dnr.wa.gov/site2/rest/services/"
#         "Public_Forest_Mgmt/WADNR_PUBLIC_RS_FRIS_Rasters/"
#         "MapServer/1/tile/{z}/{y}/{x}"
#     ),
#     name="WA DNR FRIS BA",
#     attr="Washington DNR",
#     overlay=True,
#     show=False,
#     control=True
# ).add_to(m)

## Contextual GIS layers 

In [ ]:
# Add Google Roads layer (optional)
folium.raster_layers.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=h&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Roads',
    overlay=True,
    control=True,
    show=False,
).add_to(m);

In [ ]:
# Watersheds
# Get GeoJSON from an API
huc8 = 'https://hydro.nationalmap.gov/arcgis/rest/services/wbd/FeatureServer/4/query?where=1%3D1&geometry=-124%2C45%2C-116%2C49&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&outFields=name%2Chuc8%2Careasqkm&outSR=4326&f=geojson&maxAllowableOffset=0.01'

gjson = folium.GeoJson(huc8,
                        embed=False,
                        show=False,
                        name="Watershed Sub-basins (HUC8)",
                        style_function=lambda x: {
                            'fillColor': 'none',
                            'color': 'blue',
                            'weight': 2,
                            'fillOpacity': 0
                        }).add_to(m)

folium.features.GeoJsonTooltip(fields=["name"], labels=False).add_to(gjson)
folium.features.GeoJsonPopup(fields=["name", "huc8", "areasqkm"], labels=True).add_to(gjson);


## Dynamic tiling of tiffs


for these we need a colorbar to interpret the rendered tiff...

In [ ]:
# Colorbar for titiler raster overlays
class ToggleableColorbar(MacroElement):
    _counter = 0  # Class-level counter for unique IDs

    def __init__(self, layer_name, colormap):
        super().__init__()
        self.layer_name = layer_name
        self.colormap = colormap
        # Use a unique counter-based name instead of self._name
        ToggleableColorbar._counter += 1
        self.legend_var_name = f"legend_{ToggleableColorbar._counter}"

        self._template = Template("""
        {% macro script(this, kwargs) %}
        // Initialize registry if not exists
        if (typeof window._colorbarLegends === 'undefined') {
            window._colorbarLegends = {};
        }

        var {{ this.legend_var_name }} = L.control({position: 'bottomright'});
        {{ this.legend_var_name }}.onAdd = function (map) {
            var div = L.DomUtil.create('div', 'info legend');
            div.style.backgroundColor = 'white';
            div.style.padding = '10px';
            div.innerHTML = `{{ this.colormap._repr_html_() }}`;
            return div;
        };
        // Don't add to map initially - will be added when layer is shown
        {{ this.legend_var_name }}._isOnMap = false;

        // Register this legend
        window._colorbarLegends['{{ this.layer_name }}'] = {{ this.legend_var_name }};

        {{ this._parent.get_name() }}.on('overlayadd', function(e) {
            if (e.name === '{{ this.layer_name }}') {
                // Remove all other legends from the map first
                for (var key in window._colorbarLegends) {
                    var otherLegend = window._colorbarLegends[key];
                    if (otherLegend._isOnMap) {
                        otherLegend.remove();
                        otherLegend._isOnMap = false;
                    }
                }
                // Add this legend to the map
                {{ this.legend_var_name }}.addTo({{ this._parent.get_name() }});
                {{ this.legend_var_name }}._isOnMap = true;
            }
        });

        {{ this._parent.get_name() }}.on('overlayremove', function(e) {
            if (e.name === '{{ this.layer_name }}') {
                if ({{ this.legend_var_name }}._isOnMap) {
                    {{ this.legend_var_name }}.remove();
                    {{ this.legend_var_name }}._isOnMap = false;
                }
            }
        });
        {% endmacro %}
        """)

### SOLUS

In [ ]:
tif = "https://storage.googleapis.com/solus100pub/anylithicdpt_cm_p.tif"
tiler = "https://titiler.xyz/cog/tiles/WebMercatorQuad/{z}/{x}/{y}"
virtual_tiles = f"{tiler}?url={tif}"
folium.TileLayer(tiles=virtual_tiles,
                 overlay=True,
                 control=True,
                 show=False,
                 name='SOLUS100 Soil Depth',
                 attr="USDA").add_to(m)

cm = branca.colormap.linear.Greys_06.scale(0,1)
cm.colors.reverse()
cm = branca.colormap.LinearColormap(colors=cm.colors, caption='SOLUS100 soil depth (cm)', vmin=0, vmax=200)

# Add the colormap to the map as a control
#cm.add_to(m)
# Link colorbar visibility to layer visibility
m.add_child(ToggleableColorbar("SOLUS100 Soil Depth", cm));

### YiYU Seismic Velocities

In [ ]:
def mpl_to_branca(cmap, vmin=0, vmax=1, n=256):
    """
    Convert a matplotlib colormap to a branca LinearColormap.

    Parameters
    ----------
    cmap : matplotlib colormap or str
        Colormap instance or name (e.g. 'viridis')
    vmin, vmax : float
        Data range for the branca colormap
    n : int
        Number of color samples

    Returns
    -------
    branca.colormap.LinearColormap
    """
    if isinstance(cmap, str):
        cmap = matplotlib.colormaps.get_cmap(cmap)

    colors = [
        mcolors.to_hex(cmap(i))
        for i in np.linspace(0, 1, n)
    ]

    return bcm.LinearColormap(
        colors=colors,
        vmin=vmin,
        vmax=vmax
    )

In [ ]:
# Test visualization
# import rioxarray as rxr
# da =rxr.open_rasterio('global_vs30_2.tif', overview_level=4).squeeze()
# da.plot.imshow(vmin=0, vmax=2000, cmap='viridis')

In [ ]:
# Set up scaling and tile rendering
layer_name = "Vs30"
attribution = "USGS"
colormap = "viridis"
vmin = 0
vmax = 2000
branca_cmap = mpl_to_branca(colormap, vmin=vmin, vmax=vmax)
branca_cmap.caption = "Vs30 Global (m/s)"

tif = "https://dasway.ess.washington.edu/shared/niyiyu/global_vs30_2.tif"
tiler = "https://titiler.xyz/cog/tiles/WebMercatorQuad/{z}/{x}/{y}"
virtual_tiles = f"{tiler}?url={tif}&rescale={vmin},{vmax}&colormap_name={colormap}"

# Display the colormap
branca_cmap

In [ ]:
folium.TileLayer(tiles=virtual_tiles,
                 overlay=True,
                 control=True,
                 show=False,
                 name=layer_name,
                 attr=attribution).add_to(m)

# Link colorbar visibility to layer visibility
m.add_child(ToggleableColorbar(layer_name, branca_cmap));

### Brandon Kerns precip and ERM layers

In [ ]:
## For rainfall accumulation

# Set up scaling and tile rendering
layer_name_bk = "Stage IV Precip, Dec. 2025"
attribution = "NOAA"
colormap = "viridis"
vmin = 0
vmax = 700
cmap_stage4 = mpl_to_branca(colormap, vmin=vmin, vmax=vmax)
cmap_stage4.caption = "Stage IV Precipitation (mm)"

tif_bk = "https://gaia-hazlab-map-data.s3.us-west-2.amazonaws.com/rainfall_4km.tif"
tiler_bk = "https://titiler.xyz/cog/tiles/WebMercatorQuad/{z}/{x}/{y}"
virtual_tiles_bk = f"{tiler_bk}?url={tif_bk}&rescale={vmin},{vmax}&nodata=nan&colormap_name={colormap}"

# Display the colormap
cmap_stage4

In [ ]:
folium.TileLayer(
    tiles=virtual_tiles_bk,
    overlay=True,
    control=True,
    show=False,
    name=layer_name_bk,
    attr=attribution,
    opacity=0.6
).add_to(m)

# Add the colormap to the map as a control
#cm.add_to(m)
# Link colorbar visibility to layer visibility
m.add_child(ToggleableColorbar(layer_name_bk, cmap_stage4));

In [ ]:
## For max ERM

# Set up scaling and tile rendering
layer_name_bk2 = "Max ERM, Dec. 2025"
attribution = "UW Atmospheric Sciences"
colormap = "plasma"
vmin = 0
vmax = 3
cmap_erm = mpl_to_branca(colormap, vmin=vmin, vmax=vmax)
cmap_erm.caption = "Max Extreme Rainfall Multiplier (ERM)"

tif_bk2 = "https://gaia-hazlab-map-data.s3.us-west-2.amazonaws.com/max_erm_4km.tif"
tiler_bk2 = "https://titiler.xyz/cog/tiles/WebMercatorQuad/{z}/{x}/{y}"
virtual_tiles_bk2 = f"{tiler_bk2}?url={tif_bk2}&rescale={vmin},{vmax}&nodata=nan&colormap_name={colormap}"

# Display the colormap
cmap_erm

In [ ]:
folium.TileLayer(
    tiles=virtual_tiles_bk2,
    overlay=True,
    control=True,
    show=False,
    name=layer_name_bk2,
    attr=attribution,
    opacity=0.6
).add_to(m)

# Add the colormap to the map as a control
#cm.add_to(m)
# Link colorbar visibility to layer visibility
m.add_child(ToggleableColorbar(layer_name_bk2, cmap_erm));

# Optional Plugins

- Ability to measure distances 
- Ability to draw vector shapes to export (e.g. study area AOI)

In [ ]:
# Optional plugins
from folium.plugins import Draw
from folium.plugins import MeasureControl

# Position options position : {'topleft', 'toprigth', 'bottomleft', 'bottomright'}
# https://github.com/python-visualization/folium/issues/1806
Draw(export=True, position='bottomleft').add_to(m)

m.add_child(MeasureControl(position='topleft'));

## Display and save map

In [ ]:
# Save the map to an HTML file
# Todo: use from folium.plugins.treelayercontrol import TreeLayerControl
# https://python-visualization.github.io/folium/latest/user_guide/plugins/treelayercontrol.html
folium.LayerControl(collapsed=True, draggable=True).add_to(m)

m

In [ ]:
m.save('index.html')